In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://i.pinimg.com/originals/4e/90/49/4e904958f535db5c88837a4f90aaf7ee.jpg)pinterest.com

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
df = pd.read_csv('../input/cusersmarildownloadsgermancsv/german.csv', delimiter=';', encoding = "ISO-8859-2", nrows = nRowsRead)
df.dataframeName = 'german.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
df.head()

In [ ]:
# Retrieve the remaining dataset as X and y values
X = df.drop('Creditability', axis=1).values
y = df['Creditability'].values

In [ ]:
sns.heatmap(df.corr(), square=True, cmap='RdYlGn')

plt.show()

In [ ]:
#Code by Porter97 https://www.kaggle.com/porter97/synthanic-logistic-regression/notebook

from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

# Use a Lasso Regression to find the features that are most important
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

features = df.drop('Creditability', axis=1).columns
lasso = Lasso(alpha=0.1)
lasso_coef = lasso.fit(X_train, y_train).coef_

_ = plt.plot(range(len(features)), lasso_coef)
_ = plt.xticks(range(len(features)), features, rotation=60)
_ = plt.ylabel('Coefficients')
plt.show()

In [ ]:
# Remove some columns as it didn't work without performing that step and redefine our X and y
df = df.drop('Age_years', axis=1).drop('Telephone', axis=1)
X = df.drop('Creditability', axis=1).values
y = df['Creditability'].values

#Trying to perform Logistic Regression I got an error "Failed to converge increase the number of iterations". Choices suggested by the programm to fix it: Scaling 

#(min/max/standard/whatever scale) or increase number of max_iter (iterations). Scaling didn't work. (because the only 

#snippet that I could understand was that below). The snippet worked, however the Logistic Regression DO NOT.

#Then I performed that log_reg = LogisticRegression(solver='lbfgs',class_weight='balanced', max_iter=1000) found in 
Stackoverflow : https://stackoverflow.com/questions/52670012/convergencewarning-liblinear-failed-to-converge-increase-the-number-of-iterati

#Answered by answered Jul 19 '20 at 10:35 -rishi jain -88611 gold badge1111 silver badges21


#Increasing max-iter to 1000 (default is 1000). The original code didn't mention any iteration.

The original code was only logreg = LogisticRegression()

#In resume, including max_iter allowed my LGBM to be performed and so did the ROC Curve.

![](https://kseow.com/images/Logistic_Regression/trump.jpg)https://kseow.com/logisticregression.html

In [ ]:
#Code By Rachael Tatman https://www.kaggle.com/rtatman/data-cleaning-challenge-scale-and-normalize-data
#Code by Alexis Cook https://www.kaggle.com/alexisbcook/exercise-scaling-and-normalization

from sklearn.preprocessing import minmax_scale

# set seed for reproducibility
np.random.seed(0)

# generate 1000 data points randomly drawn from an exponential distribution
df = np.random.exponential(size = 1000)

# mix-max scale the data between 0 and 1
scaled_df = minmax_scale(df)

# plot both together to compare
fig, ax=plt.subplots(1,2)
sns.histplot(df, ax=ax[0], color='y')
ax[0].set_title("Creditability Data")
sns.histplot(scaled_df, ax=ax[1])
ax[1].set_title("Scaled data")
plt.show()

In [ ]:
#Code by Porter97 https://www.kaggle.com/porter97/synthanic-logistic-regression/notebook

from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='lbfgs',class_weight='balanced', max_iter=1000)#https://stackoverflow.com/questions/52670012/convergencewarning-liblinear-failed-to-converge-increase-the-number-of-iterati

# Create a new training and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

# Perform a logistic regression on the data
#logreg = LogisticRegression() #Change that to use max-iter line above (Stackoverflow)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)

In [ ]:
#Code by Porter97 https://www.kaggle.com/porter97/synthanic-logistic-regression/notebook

from sklearn.metrics import roc_curve

# Measure the ROC curve to determine the performance of the model
y_pred_prob = log_reg.predict_proba(X_test)[:,1]

# Retrieve the False Positive and True Positive Rates, along with the threshold
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Plot the results
plt.plot([0,1], [0,1], 'k--')
plt.plot(fpr, tpr, label="Logistic Regression")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression ROC Curve')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

# Return the ROC-AUC score for our model
roc_auc_score(y_test, y_pred_prob)

In [ ]:
#Code by Porter97 https://www.kaggle.com/porter97/synthanic-logistic-regression/notebook

from sklearn.model_selection import cross_val_score

# Get the cross value scores for ROC-AUC scoring, returning an array and the mean
cv_scores = cross_val_score(log_reg, X, y, cv=10, scoring='roc_auc')

print(cv_scores)
print(np.mean(cv_scores))

#I don't even know if the code is correct. However, I could perform the ROC curve that I intended to make.

#Thank you Porter98,  Alexis Cook, Rachael Tatman and once again StackOverflow (rishi jain answer).